# Creating a Simple Agent with Tracing

In [7]:
import dotenv
import os 

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print("""Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """)

# Test OpenAI Access
print(OpenAI().responses.create(
    model=os.environ["OPENAI_DEFAULT_MODEL"],
    input='Say: We are up and running!').output_text)



We are up and running!


In [8]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent


Create a simple Nutrition Assistant Agent

In [9]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """,
)

Let's execute the Agent:

In [10]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Overall, yes—bananas are a healthy, convenient fruit.
    
    Key positives:
    - Potassium-rich (supports blood pressure and heart health)
    - Good vitamin B6 and vitamin C sources
    - Dietary fiber (helps digestion) and resistant starch when underripe
    - Easy, portable, affordable
    
    Considerations:
    - Moderate calories and natural sugars (about 14 g sugar per medium banana)
    - If you have kidney disease or need to limit potassium, check with a clinician
    
    Typical nutrition for a medium banana (~118 g):
    - Calories: ~105
    - Carbs: ~27 g (fiber ~3 g, sugar ~14 g)
    - Potassium: ~422 mg
    - Vitamin B6 and vitamin C present
    
    If you want, I can tailor advice to your goals (weight loss, muscle gain, blood pressure, or kidney health).
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more 

Streaming the answer to the screen, token by token

In [12]:
response_stream = Runner.run_streamed(
    nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are generally healthy and nutrient-dense.

- Good stuff: potassium, vitamin B6, vitamin C, fiber, and resistant starch (especially when slightly underripe).
- Benefits: supports blood pressure, digestion, energy, and gut health.
- Considerations: moderate sugar content; portion if you’re watching blood sugar or calories.
- Quick tip: 1 medium banana (~118 g) is a common healthy portion.

Want a quick plan (e.g., pre/post workout or diabetes-friendly) tailored to you?

_Good Job!_